In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
train.shape

In [ ]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
def balancing(dataframe):
    class_0,class_1 = dataframe.target.value_counts()
    df_0 = dataframe[dataframe['target']== 0]
    df_1 = dataframe[dataframe['target']== 1]
    df_1_over = df_1.sample(class_0,replace=True)
    df_train_over = pd.concat([df_0,df_1_over],axis =0)
    return df_train_over

In [ ]:
train_balanced = balancing(train)

In [ ]:
train_balanced.shape

In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

In [ ]:
train_balanced['cleanText']=train_balanced['text'].map(lambda s:preprocess(s)) 

In [ ]:
test['cleanText']=test['text'].map(lambda s:preprocess(s)) 

In [ ]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_balanced.cleanText,train_balanced.target))#convert dataframe to dataset

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test.cleanText))

In [ ]:
#shuffle the data so that we see a different ordering each epoch.
#prefetch(tf.data.AUTOTUNE)///process and prepare the next batch of data. What this will do is that when the next training step starts, 
#we don’t have to wait for the next batch of data to be prepared. 
#It will automatically be there, and this can reduce the overall training time significantly.
train_dataset = training_dataset.shuffle(1000).batch(64).prefetch(tf.data.AUTOTUNE)#shuffle(BUFFER_SIZE)
#عشان يسوي فيكتور للكلمات يحتاج ثلاث ابعاد ..و انا كان بس عندي بعدين .. الباتش حل المشكله مدري ليش

In [ ]:
test_dataset = test_ds.batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
# for example, label in val_dataset.take(9):
#   print('texts: ', example.numpy())
#   print('labels: ', label.numpy())
#   print()

In [ ]:
VOCAB_SIZE = 10000
# هنا يعطي الكلمات ارقام اكثر كلمه مكرره في قاعده البيانات يعطيها صفر و بعدين ثاني كلمه واحد و هكذا )(مومتاكده)

encoder = tf.keras.layers.TextVectorization()#max_tokens=VOCAB_SIZE) #اللي فهمته انه يرقم الكلمات و الكلمات اللي ترقيمها )(تكرارها فوق الالف الفوكاب سايز يصير يحطها واحد  )

# #output_sequence_length=250.. يخليهم كلهم نفس الطول 250 رقم في كل جمله يعطي ارقام للكلمات و الباقي اصفار



# # Make a text-only dataset (without labels), then call adapt

encoder.adapt(train_dataset.map(lambda text,label:text))

#Removing Unicode Characters

In [ ]:
# print("1287 ---> ",encoder.get_vocabulary()[1287])
# print(" 313 ---> ",encoder.get_vocabulary()[313])
# print('Vocabulary size: {}'.format(len(encoder.get_vocabulary())))

In [ ]:
#اتوقع كل الكلممات صارت حروف صغيره
# vocab = np.array(encoder.get_vocabulary())
# vocab[:120]

In [ ]:
#len(encoder.get_vocabulary())

In [ ]:
#كل الجمل صارت نفس الطول
# encoded_example = encoder(example).numpy()
# encoded_example

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    #tf.keras.layers.LSTM(64),
    tf.keras.layers.GlobalAveragePooling1D(),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

#predicted value): This is the model's prediction, i.e, a single floating-point value which either represents a logit,
#(i.e, value in [-inf, inf] when from_logits=True) or a probability (i.e, value in [0., 1.] when from_logits=False).

In [ ]:
history = model.fit(train_dataset, epochs=30)
#                     validation_data=val_dataset,
#                     validation_steps=30)

In [ ]:
#model.evaluate(val_dataset)

In [ ]:
pred = model.predict(test_dataset)

In [ ]:
preds=[]
for i in pred :
    if i > 0.5 :
        i = 1
    if i < 0.5:
        i=0
    preds.append(i)

In [ ]:
# for text in test_ds.take():
#     print(text)

In [ ]:
test.insert(4,"target", preds)

In [ ]:
test.head()

In [ ]:
output = test[['id','target']]

In [ ]:
output.to_csv('submission.csv', index=False)